<a href="https://colab.research.google.com/github/valerio-unifei/ECAA07/blob/main/ECAA07_08_API_RESTful.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Atenção:**

Este código funciona apenas no VSCode no Ubuntu nas máquinas do LEC.

# API RESTful

**Representational State Transfer (REST)**: Transferência Representacional de Estado

Uma **API RESTful** (também conhecida como API REST) é uma interface de programação de aplicativos (API ou API Web) que está em conformidade com as restrições do estilo de arquitetura REST e permite a interação com serviços Web RESTful.

Uma **API** é um conjunto de definições e protocolos para construir e integrar software de aplicação. Às vezes, é chamado de contrato entre um provedor de informações e um usuário de informações – estabelecendo o conteúdo exigido do consumidor (a chamada) e o conteúdo exigido pelo produtor (a resposta). Por exemplo, o design da API para um serviço meteorológico pode especificar que o usuário forneça um CEP e que o produtor responda com uma resposta em duas partes, sendo a primeira a temperatura alta e a segunda a baixa.  

Em outras palavras, se você deseja interagir com um computador ou sistema para recuperar informações ou executar uma função, uma API o ajuda a comunicar o que deseja a esse sistema para que ele possa entender e atender à solicitação.

Você pode pensar em uma API como um mediador entre os usuários ou clientes e os recursos ou serviços da Web que eles desejam obter. É também uma forma de uma organização compartilhar recursos e informações enquanto mantém a segurança, o controle e a autenticação — determinando quem tem acesso a quê.

<img src="https://cdn-ajfbi.nitrocdn.com/GuYcnotRkcKfJXshTEEKnCZTOtUwxDnm/assets/static/optimized/rev-5f831ce/wp-content/uploads/2020/01/rest.png">

# Iniciando a Aplicação Web

## Bibliotecas para Python

In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn

## Instalando MongoDB

In [ ]:
!apt install mongodb >log
!service mongodb start

## Preparando Banco

In [ ]:
from pymongo import MongoClient
# conectando ao MongoDB Local
client = MongoClient()
# criando banco
db = client.get_database('ecaa07')
# criando coleção
col = db.get_collection('medidas')
db.name, col.name

### Inserindo Dados

Dados sintéticos a partir de "2010-08-01 12:00:00":

In [ ]:
from random import seed, randint, random
from datetime import datetime, timedelta
matr = int(input('Forneça sua matrícula: '))
seed(matr)
# gerando medidas sintéticas
medidas = []
for i in range(randint(50,100)):
  med = {'dispositivo':1,'instante':datetime(2010,8,1,12,0,0)+timedelta(minutes=i)}
  for c in range(randint(3,6)):
    med[f'sensor_{c}'] = random()*1000.0
  medidas.append(med)
# inserindo no banco
res = col.insert_many(medidas)
print('Sucesso:',res.acknowledged,', inseridos:',len(res.inserted_ids))

# Configurando API RESTful

In [ ]:
from fastapi import FastAPI # cria o serviço de Rest API
from fastapi.responses import JSONResponse # converte valores para JSON
app = FastAPI()
# domínio inicial da aplicação
@app.get('/')
def home():
  return f'Aplicação de {matr}'

## Listando Medidas

In [ ]:
@app.get('/medidas')
def medidas():
  client = MongoClient()
  # TODO: faça a leitura de todas as medidas do banco
  json = []
  return json

## Filtrando por Dispositivo

In [ ]:
@app.post('/dispositivo')
def dispositivo(nome:str):
  client = MongoClient()
  # TODO: faça uma query para ler medidas do dispositivo pedido
  json = []
  return json

## Filtrando por Intervalo de Tempo

In [ ]:
@app.post('/intervalo')
def intervalo(inicio:str, fim:str):
  client = MongoClient()
  # TODO: faça uma query para ler medidas de um intervalo de tempo
  json = []
  return json

# Ativando o Serviço e Gerando Link

Vá no site: https://dashboard.ngrok.com/login
- Crie uma conta.
- Gere um token de acesso: https://dashboard.ngrok.com/get-started/your-authtoken
- Insira o token no código abaixo

In [ ]:
from pyngrok import ngrok, conf
conf.get_default().auth_token = 'SEU_TOKEN_AQUI'

import nest_asyncio #permite execução de servidor em loop no Colab
from pyngrok import ngrok #dá um nome de DNS real para a aplicação
import uvicorn #gestor de aplicação web

connection_string = ngrok.connect(8000).public_url

print()
print('Abra no navegador:', connection_string + '/docs')
print()
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Teste a aplicação no link gerado acima.